In [5]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_csv('/home/jupyter-vladperesad/files/THE QUERY-.csv')
df

In [63]:
df_2 = df.drop(columns=['student_name',
                        'group_name',
                        'date'])

In [64]:
df_2 = df \
    .drop(columns=['student_name',
                    'group_name',
                    'date']) \
    .groupby('student_id') \
    .pct_change(fill_method=None)*100

In [65]:
df_2.dropna(axis=0, how='all', inplace=True)

In [67]:
final_table = df.join(df_2, how='inner', rsuffix= '_pct')